# Pre-processing Data

## 1. Importing Data

In [1]:
import pandas as pd
data=pd.read_csv('credit_ds_v3.csv')
data.head()

,X,number_of_cards,outstanding,credit_limit,bill,total_cash_usage,total_retail_usage,remaining_bill,branch_code,payment_ratio,...,default_flag,years_since_card_issuing,total_usage,remaining_bill_per_number_of_cards,remaining_bill_per_limit,total_usage_per_limit,total_3mo_usage_per_limit,total_6mo_usage_per_limit,utilization_3month,utilization_6month
0,2,2,268691,10000000.0,254564.0,0.0,1012.0,0.0,A,0.00,...,0,0.750000,1012.0,0.0,0.000000,0.000101,0.000000,0.000000,0.004232,0.000300
1,3,3,6769149,28000000.0,4159779.0,0.0,0.0,0.0,A,100.00,...,0,10.750000,0.0,0.0,0.000000,0.000000,0.040518,0.047703,0.249389,NaN
2,5,2,9402085,10000000.0,6099283.0,0.0,2666558.0,5951865.0,A,95.99,...,0,1.666667,2666558.0,2975932.5,0.595186,0.266656,0.323027,0.131162,0.707865,0.336571
3,6,2,6227439,80000000.0,2081248.0,0.0,3690250.0,4613435.0,A,0.00,...,0,10.250000,3690250.0,2306717.5,0.057668,0.046128,0.017184,0.117513,0.038964,0.181863
4,7,2,3906290,4000000.0,2043682.0,0.0,230400.0,3314046.0,A,18.47,...,0,4.666667,230400.0,1657023.0,0.828511,0.057600,0.018750,0.166667,1.014776,0.977231


## 2. Checking total number of "null" value in every columns

In [6]:
data.isnull().sum()

X                                        0
number_of_cards                          0
outstanding                              0
credit_limit                             0
bill                                     0
total_cash_usage                        45
total_retail_usage                       0
remaining_bill                           0
branch_code                            195
payment_ratio                            0
overlimit_percentage                    26
payment_ratio_3month                     0
payment_ratio_6month                     0
delinquency_score                       88
default_flag                             0
years_since_card_issuing                 0
total_usage                              0
remaining_bill_per_number_of_cards       0
remaining_bill_per_limit                 0
total_usage_per_limit                    0
total_3mo_usage_per_limit                0
total_6mo_usage_per_limit                0
utilization_3month                       0
utilization

## 3. filling null value in every columns

### 3.1 Filling null value in total_cash_usage columns

In [3]:
#checking the value range and distribution, so I will expect the 0.0 value replace the dominant NaN value
data['total_cash_usage'].value_counts()

0.0          14864
400000.0        60
1000000.0       51
300000.0        50
200000.0        48
             ...  
3400000.0        1
2080000.0        1
1194665.0        1
4200000.0        1
1150000.0        1
Name: total_cash_usage, Length: 94, dtype: int64

#### replacing missing value with linear regression

In [4]:
data['total_cash_usage'].interpolate(method='linear', inplace=True, limit_direction="both")

### 3.2 Filling null value in branch_code columns

In [5]:
data['branch_code'].value_counts()

A    8705
F    1697
B    1351
I     991
E     649
G     566
J     410
H     367
C     334
D     205
K     175
Name: branch_code, dtype: int64

In [6]:
unique = data.groupby('branch_code')['default_flag'].nunique()
unique

branch_code
A    2
B    2
C    2
D    2
E    2
F    2
G    2
H    2
I    2
J    2
K    2
Name: default_flag, dtype: int64

- for any branch, always found fraud. So, I'm using every branch name to fill the missing value based on probability
- assuming there is'nt other branch, all branch already listed

In [7]:
null = data['branch_code'].isnull()
data['branch_code'].loc[null] = data['branch_code'].dropna().sample(null.sum(), random_state=1).values

C:\Users\faisa\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


#### Label Encoding for column branch_code

In [8]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
encoder.fit(data['branch_code'])
transformed = encoder.transform(data['branch_code'])
df = pd.DataFrame(transformed)
data = pd.concat([data, df], axis=1).drop(['branch_code'], axis=1)

### 3.3 Filling null value in overlimit_percentage columns

In [9]:
data['overlimit_percentage'].value_counts()

0.00     11481
16.00       29
10.90       25
12.80       23
14.90       23
         ...  
17.06        1
31.80        1
7.64         1
9.60         1
0.51         1
Name: overlimit_percentage, Length: 1222, dtype: int64

#### filling the missing value with linear regression

In [10]:
data['overlimit_percentage'].interpolate(method='linear', inplace=True, limit_direction="both")

### 3.4 Filling null value in delinquency_score columns

In [11]:
data['delinquency_score'].value_counts()

0.0    15394
5.0       42
4.0       34
3.0       34
1.0       29
2.0       24
Name: delinquency_score, dtype: int64

#### check the distribution of deliquency score on default_flag

In [12]:
check_distribution= data.groupby(['delinquency_score','default_flag']).size().reset_index().rename(columns={0:'count'})
check_distribution

,delinquency_score,default_flag,count
0,0.0,0,14114
1,0.0,1,1280
2,1.0,0,22
3,1.0,1,7
4,2.0,0,3
5,2.0,1,21
6,3.0,0,2
7,3.0,1,32
8,4.0,1,34
9,5.0,1,42


- assuming deliquency score range is 0-5
- from this data only default flag 1 occur from any deliquency score
- meanwhile default flag 0 only occur from deliquency score 0 - 3
- hence, I'm filling the missing value based on this distribution

In [13]:
null = data['delinquency_score'].isnull()
# data['delinquency_score'][data['default_flag']==0].loc[null] = data['delinquency_score'][data['default_flag']==0].dropna().sample(null.sum(), random_state=1).values

In [14]:
counts= check_distribution['count'][check_distribution['default_flag']==0]
counts= counts.to_numpy()

delScore= check_distribution['delinquency_score'][check_distribution['default_flag']==0]
delScore= delScore.to_numpy()

import numpy as np
pool= np.repeat((delScore), (counts))
np.random.shuffle(pool)
choosen= np.random.choice(pool, sum(null), replace=False)

data['delinquency_score'].loc[(data['delinquency_score'].isnull())]= data['delinquency_score'].loc[(data['delinquency_score'].isnull())].dropna()
data['delinquency_score'].loc[(data['delinquency_score'].isnull())]= choosen

C:\Users\faisa\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


### 3.4 Filling null value in utilization_6month columns

In [15]:
data['utilization_6month'].value_counts()

0.000000    229
1.010000    105
1.040000     75
1.020000     73
1.090000     73
           ... 
0.674108      1
0.147862      1
0.769356      1
0.733093      1
0.049121      1
Name: utilization_6month, Length: 3643, dtype: int64

In [16]:
null = data['utilization_6month'].isnull()
data['utilization_6month'].interpolate(method='linear', inplace=True, limit_direction="both")

In [17]:
# data.to_csv("credit_ds_v3clean.csv")